In [1]:
import pandas as pd

In [2]:
weather_data_file = '../data/NOAA_weather_data/1992-1997_california_box_all.csv'
fire_data_file = '../data/USDA_wildfire_data/regridded_california_fires.csv'
training_data_file = '../data/training_data/1992-1997_training_data_box.csv'

# Parallelization options
N_PROCESSES = 15
JOBS_PER_PROCESS = 1

In [3]:
weather = pd.read_csv(weather_data_file)
weather['time'] = pd.to_datetime(weather['time'])
weather['date'] = weather['time'].dt.date
weather['date'] = pd.to_datetime(weather['date'])

fires = pd.read_csv(fire_data_file)
fires['ignition'] = 1
fires['date'] = pd.to_datetime(fires['date'])

In [4]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24375040 entries, 0 to 24375039
Data columns (total 23 columns):
time         datetime64[ns]
lat          float64
lon          float64
air.sfc      float64
air.2m       float64
apcp         float64
crain        float64
rhum.2m      float64
dpt.2m       float64
pres.sfc     float64
uwnd.10m     float64
vwnd.10m     float64
veg          float64
dlwrf        float64
dswrf        float64
lcdc         float64
hcdc         float64
mcdc         float64
hpbl         float64
prate        float64
vis          float64
ulwrf.sfc    float64
date         datetime64[ns]
dtypes: datetime64[ns](2), float64(21)
memory usage: 4.2 GB


In [5]:
fires.head()

,lat,lon,date,time,size,size_class,ignition
0,39.93427,-121.1598,2005-02-02,1300.0,0.10,A,1
1,38.85770,-120.5208,2004-05-12,845.0,0.25,A,1
2,39.13992,-120.5874,2004-05-31,1921.0,0.10,A,1
3,38.62627,-120.0940,2004-06-28,1600.0,0.10,A,1
4,38.62627,-120.0940,2004-06-28,1600.0,0.10,A,1


In [6]:
fires.tail()

,lat,lon,date,time,size,size_class,ignition
189545,40.32918,-122.4153,2015-09-26,1726.0,0.01,A,1
189546,37.62744,-120.9710,2015-10-05,126.0,0.20,A,1
189547,37.62744,-120.9710,2015-05-02,2052.0,0.10,A,1
189548,37.62744,-120.9710,2015-10-14,2309.0,2.00,B,1
189549,34.15059,-116.7372,2015-03-14,2128.0,0.10,A,1


In [7]:
training_data = pd.merge(weather, fires, on=['lat', 'lon', 'date'], how='left')

In [8]:
training_data.head()

,time_x,lat,lon,air.sfc,air.2m,apcp,crain,rhum.2m,dpt.2m,pres.sfc,...,mcdc,hpbl,prate,vis,ulwrf.sfc,date,time_y,size,size_class,ignition
0,1992-01-01,31.63322,-125.3449,288.66638,289.13074,0.367188,1.0,77.1875,285.05447,101573.56,...,17.0,802.157840,0.000037,20007.979,382.2500,1992-01-01,NaN,NaN,NaN,NaN
1,1992-01-01,31.90358,-125.4247,288.66638,289.13074,0.304688,1.0,77.1875,285.05447,101573.56,...,6.0,538.557800,0.000027,20007.979,382.2500,1992-01-01,NaN,NaN,NaN,NaN
2,1992-01-01,31.70029,-125.0272,288.66638,289.06824,0.304688,1.0,78.8125,285.31230,101573.56,...,4.0,439.757800,0.000027,20007.979,382.3125,1992-01-01,NaN,NaN,NaN,NaN
3,1992-01-01,31.97103,-125.1057,288.41638,288.88074,0.210938,1.0,79.6875,285.33572,101573.56,...,0.0,63.857810,0.000017,20007.979,381.1875,1992-01-01,NaN,NaN,NaN,NaN
4,1992-01-01,32.24205,-125.1849,288.22888,288.69324,0.210938,1.0,81.6875,285.49980,101573.56,...,0.0,75.157814,0.000017,20007.979,380.0625,1992-01-01,NaN,NaN,NaN,NaN


In [9]:
training_data['air.sfc'].isnull().sum()

0

In [10]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24472488 entries, 0 to 24472487
Data columns (total 27 columns):
time_x        datetime64[ns]
lat           float64
lon           float64
air.sfc       float64
air.2m        float64
apcp          float64
crain         float64
rhum.2m       float64
dpt.2m        float64
pres.sfc      float64
uwnd.10m      float64
vwnd.10m      float64
veg           float64
dlwrf         float64
dswrf         float64
lcdc          float64
hcdc          float64
mcdc          float64
hpbl          float64
prate         float64
vis           float64
ulwrf.sfc     float64
date          datetime64[ns]
time_y        float64
size          float64
size_class    object
ignition      float64
dtypes: datetime64[ns](2), float64(24), object(1)
memory usage: 5.1+ GB


In [11]:
training_data.rename(columns={'time_x':'weather_bin_time','time_y':'fire_discovery_time'}, inplace=True)
training_data.drop(['date'], axis=1, inplace=True)
training_data.head()

,weather_bin_time,lat,lon,air.sfc,air.2m,apcp,crain,rhum.2m,dpt.2m,pres.sfc,...,hcdc,mcdc,hpbl,prate,vis,ulwrf.sfc,fire_discovery_time,size,size_class,ignition
0,1992-01-01,31.63322,-125.3449,288.66638,289.13074,0.367188,1.0,77.1875,285.05447,101573.56,...,100.0,17.0,802.157840,0.000037,20007.979,382.2500,NaN,NaN,NaN,NaN
1,1992-01-01,31.90358,-125.4247,288.66638,289.13074,0.304688,1.0,77.1875,285.05447,101573.56,...,100.0,6.0,538.557800,0.000027,20007.979,382.2500,NaN,NaN,NaN,NaN
2,1992-01-01,31.70029,-125.0272,288.66638,289.06824,0.304688,1.0,78.8125,285.31230,101573.56,...,100.0,4.0,439.757800,0.000027,20007.979,382.3125,NaN,NaN,NaN,NaN
3,1992-01-01,31.97103,-125.1057,288.41638,288.88074,0.210938,1.0,79.6875,285.33572,101573.56,...,98.0,0.0,63.857810,0.000017,20007.979,381.1875,NaN,NaN,NaN,NaN
4,1992-01-01,32.24205,-125.1849,288.22888,288.69324,0.210938,1.0,81.6875,285.49980,101573.56,...,88.0,0.0,75.157814,0.000017,20007.979,380.0625,NaN,NaN,NaN,NaN


In [12]:
training_data.describe()

,lat,lon,air.sfc,air.2m,apcp,crain,rhum.2m,dpt.2m,pres.sfc,uwnd.10m,...,lcdc,hcdc,mcdc,hpbl,prate,vis,ulwrf.sfc,fire_discovery_time,size,ignition
count,2.447249e+07,2.447249e+07,2.447249e+07,2.447249e+07,2.447249e+07,2.447249e+07,2.447249e+07,2.447249e+07,2.447249e+07,2.447249e+07,...,2.447249e+07,2.447249e+07,2.447249e+07,2.447249e+07,2.447249e+07,2.447249e+07,2.447249e+07,145832.000000,417928.000000,417928.0
mean,3.703913e+01,-1.193081e+02,2.881954e+02,2.871796e+02,1.303836e-01,7.508313e-02,6.058680e+01,2.774135e+02,9.262407e+04,1.643090e+00,...,1.955961e+01,1.942070e+01,1.893989e+01,9.849271e+02,1.193938e-05,1.842825e+04,3.911801e+02,1367.416424,43.521856,1.0
std,3.308083e+00,3.559808e+00,1.155349e+01,8.772353e+00,6.694179e-01,2.635254e-01,2.835222e+01,8.480812e+00,8.523757e+03,2.889355e+00,...,3.303740e+01,3.050934e+01,3.348661e+01,9.625278e+02,7.059893e-05,4.553001e+03,6.496561e+01,478.308822,995.859630,0.0
min,3.153068e+01,-1.254445e+02,2.416880e+02,2.460385e+02,0.000000e+00,0.000000e+00,1.000000e+00,1.921248e+02,6.729253e+04,-1.900815e+01,...,0.000000e+00,0.000000e+00,0.000000e+00,6.186738e+01,-4.760918e-05,4.351490e+00,1.950000e+02,0.000000,0.100000,1.0
25%,3.414266e+01,-1.223950e+02,2.815104e+02,2.825154e+02,0.000000e+00,0.000000e+00,3.375251e+01,2.706477e+02,8.452207e+04,-3.400774e-01,...,0.000000e+00,0.000000e+00,0.000000e+00,3.321681e+02,-2.286778e-06,2.000665e+04,3.567500e+02,1100.000000,0.100000,1.0
50%,3.693702e+01,-1.193113e+02,2.879692e+02,2.874987e+02,0.000000e+00,0.000000e+00,6.473834e+01,2.778078e+02,9.383542e+04,1.612701e+00,...,0.000000e+00,5.000000e+00,0.000000e+00,6.164089e+02,0.000000e+00,2.000811e+04,3.808125e+02,1417.000000,0.200000,1.0
75%,3.987430e+01,-1.162261e+02,2.921091e+02,2.911141e+02,7.812500e-03,0.000000e+00,8.940763e+01,2.851461e+02,1.014474e+05,3.486813e+00,...,3.200000e+01,2.200000e+01,2.400000e+01,1.373996e+03,3.961025e-06,2.001015e+04,4.068125e+02,1701.000000,1.000000,1.0
max,4.299324e+01,-1.131324e+02,3.365820e+02,3.233755e+02,5.241433e+01,1.000000e+00,1.000312e+02,3.040451e+02,1.037565e+05,2.032946e+01,...,1.000000e+02,1.000000e+02,1.000000e+02,8.151611e+03,6.394466e-03,2.002440e+04,7.135000e+02,2359.000000,106668.000000,1.0


In [ ]:
training_data.to_csv(training_data_file, index=False)